In [4]:
%pip install llama-index-llms-huggingface

In [5]:
!pip install llama-index

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

In [7]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [8]:
documents = SimpleDirectoryReader("./data/").load_data()

In [9]:
print(documents)

[Document(id_='db5150de-4f44-411d-a5d6-cd7cd5b5d51a', embedding=None, metadata={'page_label': '1', 'file_name': 'rag.pdf', 'file_path': '/content/data/rag.pdf', 'file_type': 'application/pdf', 'file_size': 1662567, 'creation_date': '2025-06-27', 'last_modified_date': '2025-06-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='1\nRetrieval-Augmented Generation for Large\nLanguage Models: A Survey\nYunfan Gaoa, Yun Xiongb, Xinyu Gao b, Kangxiang Jia b, Jinliu Pan b, Yuxi Bic, Yi Dai a, Jiawei Sun a, Meng\nWangc, and Haofen Wang a,c\naShanghai Research Institute for Intelligent Autonomous Systems, Tongji University\nbShanghai Key Laborator

In [10]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [11]:
!pip install -q transformers accelerate huggingface_hub

In [12]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

In [13]:
from huggingface_hub import login
login(token="")

HuggingFaceLLM is a wrapper (or connector) that allows you to use Hugging Face models (like Mistral, Falcon, LLaMA, etc.) within a framework like LlamaIndex or LangChain for tasks such as question-answering, summarization, RAG, and chatbots.

In [14]:
import torch
import os
from llama_index.llms.huggingface import HuggingFaceLLM

# Set token (you already did this above)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={
        "torch_dtype": torch.float16,
        "use_auth_token": os.environ["HUGGINGFACEHUB_API_TOKEN"]
    }
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [16]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model =HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

In [18]:
from llama_index.core import VectorStoreIndex, Settings  # ✅ use Settings now

# Set global/default settings
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024


In [20]:
index = VectorStoreIndex.from_documents(documents)

In [21]:
query_engine = index.as_query_engine()

In [24]:
query_engine.query("what is rag?")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response(response="RAG stands for Retrieval-Augmentation-Generation. It is a paradigm for information retrieval and generation that involves three main stages: indexing, retrieval, and generation. The goal of indexing is to enhance the quality of the content being indexed, while the goal of retrieval is to make the user's original question clearer and more suitable for the retrieval task. The post-retrieval process involves integrating the retrieved information effectively with the query. The Modular RAG architecture advances beyond the previous two RAG paradigms, offering enhanced adaptability and versatility. It incorporates diverse strategies for improving its components, such as adding a search module for similarity searches and refining the retriever through fine-tuning.", source_nodes=[NodeWithScore(node=TextNode(id_='164ff33f-4a45-4747-b92f-ed7cfadcfc1f', embedding=None, metadata={'page_label': '4', 'file_name': 'rag.pdf', 'file_path': '/content/data/rag.pdf', 'file_type': 'appl